In [52]:
data_dir = 'data'
result_dir = 'result'

# Import

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [54]:
af = 'vv'
pf = '__1__'

Xval = np.load(f'{data_dir}/{af}3.v.X.{pf}.npy', allow_pickle=True)
y_val = np.load(f'{data_dir}/{af}3.v.y.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}3.fts_cols.{pf}.txt').readlines()]

In [55]:
# """ split train test """
# from sklearn.model_selection import train_test_split

# Xtrain, Xtest, y_train, y_test = train_test_split(Xval, y_val, test_size=0.3, random_state=42)

# print(Xtrain.shape, Xtest.shape)
# print(len(np.where(y_train == 1)[0]), len(np.where(y_test == 1)[0]))

# # print(list(np.where(y_train == 1)[0]))
# # print(list(np.where(y_test == 1)[0]))
# print(np.where(y_train == 1)[0])
# print(np.where(y_test == 1)[0])

In [56]:
# # Xtrain = Xtrain[100058:,:]
# Xtrain = Xtrain[300000:,:]
# Xtest = Xtest[100100:,:]
# y_train = y_train[300000:]
# y_test = y_test[100100:]

# print(Xtrain.shape, Xtest.shape)
# print(len(np.where(y_train == 1)[0]), len(np.where(y_test == 1)[0]))

In [57]:
# print(list(np.where(y_val == 1)[0]))
# print(np.where(y_val == 1)[0])

In [58]:
Xval = Xval[462995:,:]
y = y_val[462995:]

In [59]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

# Test and evaluate

In [60]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    #? relabel to use classification metrics
    y_pred[y_pred != -1] = 0
    y_pred[y_pred == -1] = 1

    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")
    return roc_auc

In [1]:
import pickle

def save_result(model, X, y_pred, scores, model_name, selected_fts_names, roc_auc, expname='', config=None):
    ra = '{:.2f}'.format(round(roc_auc * 100, 2))
    # if ra <= 70: #? not good enough to even bother
    #     return
    
    #? save model
    pickle.dump(model, open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.model.pkl','wb'))

    #? save prediction
    np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.result.t.y_pred.npy', y_pred)
    np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.result.__{ra}__.t.scores.npy', scores)

    #? save transformed X as well
    np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.data.X.npy', X)

    #? save fts
    open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.fts.txt', 'w').write('\n'.join(selected_fts_names))
    
    #? save config
    if config is not None:
        open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.config.txt', 'w').write(config)

In [62]:
from sklearn.neighbors import LocalOutlierFactor

def test_param(x, y, n_neighbors, contamination, metric='minkowski', p=2):
    model = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination, metric=metric, p=p, n_jobs=10)

    """ train """
    y_pred = model.fit_predict(x)

    """ evaluate """
    print('Evaluate on test set')
    roc_auc = evaluate(y, y_pred)

    # scores = model.score_samples(x_test)
    scores = model.negative_outlier_factor_

    del x
    # del y_test_pred, y_train_pred
    # del model

    return model, y_pred, roc_auc, scores

## Exp 01: Run on original features (0-8)

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`

In [63]:
""" select features """
selected_fts_names__01 = fts_names[0:8]
', '.join(selected_fts_names__01)

'Dur, sTos, dTos, Sport, Dport, PktsPerSec, BytesPerSec, SrcBytesPerSec'

In [64]:
X__01 = Xval[:,:]

print(X__01.shape)

(301637, 63)


In [49]:
n_neighbors = 40
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[448659   4465]
 [ 11326    182]]
              precision    recall  f1-score   support

           0     0.9754    0.9901    0.9827    453124
           1     0.0392    0.0158    0.0225     11508

    accuracy                         0.9660    464632
   macro avg     0.5073    0.5030    0.5026    464632
weighted avg     0.9522    0.9660    0.9589    464632

0.5029806351541762


In [65]:
n_neighbors = 40
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[292276   2812]
 [  6344    205]]
              precision    recall  f1-score   support

           0     0.9788    0.9905    0.9846    295088
           1     0.0679    0.0313    0.0429      6549

    accuracy                         0.9696    301637
   macro avg     0.5234    0.5109    0.5137    301637
weighted avg     0.9590    0.9696    0.9641    301637

0.510886564098269


In [66]:
n_neighbors = 20
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[292205   2883]
 [  6415    134]]
              precision    recall  f1-score   support

           0     0.9785    0.9902    0.9843    295088
           1     0.0444    0.0205    0.0280      6549

    accuracy                         0.9692    301637
   macro avg     0.5115    0.5053    0.5062    301637
weighted avg     0.9582    0.9692    0.9636    301637

0.5053455860900432


In [46]:
n_neighbors = 30
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[126090   1258]
 [  1867     75]]
              precision    recall  f1-score   support

           0     0.9854    0.9901    0.9878    127348
           1     0.0563    0.0386    0.0458      1942

    accuracy                         0.9758    129290
   macro avg     0.5208    0.5144    0.5168    129290
weighted avg     0.9715    0.9758    0.9736    129290

0.5143707680410065


In [43]:
n_neighbors = 50
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[126142   1206]
 [  1862     80]]
              precision    recall  f1-score   support

           0     0.9855    0.9905    0.9880    127348
           1     0.0622    0.0412    0.0496      1942

    accuracy                         0.9763    129290
   macro avg     0.5238    0.5159    0.5188    129290
weighted avg     0.9716    0.9763    0.9739    129290

0.5158622656530544


In [47]:
n_neighbors = 60
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[126120   1228]
 [  1858     84]]
              precision    recall  f1-score   support

           0     0.9855    0.9904    0.9879    127348
           1     0.0640    0.0433    0.0516      1942

    accuracy                         0.9761    129290
   macro avg     0.5248    0.5168    0.5198    129290
weighted avg     0.9716    0.9761    0.9739    129290

0.5168057542851433


In [ ]:
# n_neighbors = 60
# contamination = 0.006
# model__01, lof_outlier_factor__01, y_test_pred__01, roc_auc__01 = test_param(X_train__01, X_test__01, y_train, y_test, n_neighbors=n_neighbors, contamination=contamination)

# save_result(model__01, X_train__01, X_test__01, y_test_pred__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[126642    706]
 [  1896     46]]
              precision    recall  f1-score   support

           0     0.9852    0.9945    0.9898    127348
           1     0.0612    0.0237    0.0341      1942

    accuracy                         0.9799    129290
   macro avg     0.5232    0.5091    0.5120    129290
weighted avg     0.9714    0.9799    0.9755    129290

0.5090715283213829


In [52]:
n_neighbors = 70
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

In [48]:
n_neighbors = 80
contamination = 0.01
model__01, y_pred__01, roc_auc__01, scores__01 = test_param(X__01, y, n_neighbors=n_neighbors, contamination=contamination)

save_result(model__01, X__01, y_pred__01, scores__01, 'lof', selected_fts_names__01, roc_auc__01, 'exp01_fts-0-8', f'n_neighbors={n_neighbors}\ncontamination={contamination}')

Evaluate on test set
[[126065   1283]
 [  1862     80]]
              precision    recall  f1-score   support

           0     0.9854    0.9899    0.9877    127348
           1     0.0587    0.0412    0.0484      1942

    accuracy                         0.9757    129290
   macro avg     0.5221    0.5156    0.5180    129290
weighted avg     0.9715    0.9757    0.9736    129290

0.515559944454449


In [ ]:
del X_train__01, X_test__01